In [1]:
import pandas as pd 
import numpy as np  
from sklearn.model_selection import train_test_split  # Para dividir los datos en conjuntos de entrenamiento, validación y prueba

In [29]:
df_games=pd.read_parquet("Merge_games_items.parquet")
df_reviews=pd.read_csv("Merge_reviews_items.csv")

In [30]:
# Eliminar columnas en df_games
columns_to_drop_games = ['release_date', 'user_id', 'playtime_forever', 'item_name']
df_games = df_games.drop(columns=columns_to_drop_games)

# Eliminar columnas en df_reviews
columns_to_drop_reviews = ['user_id', 'posted', 'sentiment_analysis']
df_reviews = df_reviews.drop(columns=columns_to_drop_reviews)


In [36]:
# Eliminar filas duplicadas en df_games
df_games = df_games.drop_duplicates()

# Restablecer los índices después de eliminar duplicados
df_games = df_games.reset_index(drop=True)


In [51]:
df_reviews=df_reviews.dropna()
df_reviews = df_reviews.drop_duplicates()
df_reviews = df_reviews.reset_index(drop=True)

In [53]:
df_games

,genres,app_name,item_id
0,"['Action', 'Indie', 'Racing']",Carmageddon Max Pack,282010
1,['Action'],Half-Life,70
2,['Strategy'],Disciples II: Gallean's Return,1640
3,['Strategy'],Disciples II: Rise of the Elves,1630
4,"['Action', 'Indie', 'RPG']",The Ship: Murder Party,2400
...,...,...,...
8069,"['Action', 'Casual', 'Indie']",Chicken Shoot Gold,259340
8070,['Action'],Day of Defeat,30
8071,"['Strategy', 'RPG', 'Indie']",Geneforge 2,200980
8072,['Action'],Unreal Tournament 2004: Editor's Choice Edition,13230


In [58]:
df_games

,genres,app_name,item_id,content
742,"['Action', 'Strategy']",Guardians of Middle-earth,111900,"['Action', 'Strategy'] Guardians of Middle-earth"
7930,"['Adventure', 'RPG']","Precipice of Darkness, Episode One",18000,"['Adventure', 'RPG'] Precipice of Darkness, Ep..."
3091,"['Indie', 'Simulation', 'Strategy']",One Night Two Crazies,509290,"['Indie', 'Simulation', 'Strategy'] One Night ..."
3126,"['Action', 'Indie', 'Early Access']",Starr Mazer: DSP,462100,"['Action', 'Indie', 'Early Access'] Starr Maze..."
4173,"['Indie', 'RPG']",To Ash,453950,"['Indie', 'RPG'] To Ash"
...,...,...,...,...
5802,['Action'],Raptor: Call of The Shadows - 2015 Edition,336060,['Action'] Raptor: Call of The Shadows - 2015 ...
5195,['Adventure'],Airport Madness 4,387850,['Adventure'] Airport Madness 4
3337,"['Action', 'Adventure']",GIGA WRECKER,454410,"['Action', 'Adventure'] GIGA WRECKER"
6100,"['Action', 'Indie', 'Racing', 'Simulation', 'E...",Homebrew - Vehicle Sandbox,325420,"['Action', 'Indie', 'Racing', 'Simulation', 'E..."


In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Combina los géneros y nombres en una sola columna para el cálculo de similitud
df_games['content'] = df_games['genres'] + ' ' + df_games['app_name']

# Utiliza TF-IDF Vectorizer para convertir el texto en vectores numéricos
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_games['content'])

# Calcula la similitud de coseno entre los juegos basados en sus vectores TF-IDF
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)


In [60]:
def get_recommendations(item_id, cosine_sim=cosine_sim):
    # Obtén el índice del juego en función de su item_id
    idx = df_games.index[df_games['item_id'] == item_id].tolist()
    
    if not idx:
        return []  # Si no se encuentra el item_id, retorna una lista vacía
    
    idx = idx[0]  # Tomamos el primer índice si hay múltiples coincidencias
    
    # Obtén las puntuaciones de similitud de contenido para el juego en cuestión
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Ordena los juegos por similitud de contenido en orden descendente
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Obtén los índices de los juegos más similares
    similar_indices = [i[0] for i in sim_scores]
    
    # Devuelve los nombres de los juegos más similares como recomendaciones
    return df_games['app_name'].iloc[similar_indices]

# Ejemplo de uso: obtener recomendaciones para un juego con item_id específico
recommendations = get_recommendations('game123')


In [61]:
def precision_recall(recommended_items, relevant_items):
    # Calcular la intersección entre los elementos recomendados y los relevantes
    intersection = set(recommended_items) & set(relevant_items)
    
    # Calcular la precisión (número de elementos relevantes recomendados / número total recomendados)
    precision = len(intersection) / len(recommended_items) if len(recommended_items) > 0 else 0.0
    
    # Calcular el recall (número de elementos relevantes recomendados / número total de elementos relevantes)
    recall = len(intersection) / len(relevant_items) if len(relevant_items) > 0 else 0.0
    
    return precision, recall

# Ejemplo de uso:
# Supongamos que 'recommended_items' son los juegos recomendados por el modelo
# y 'relevant_items' son los juegos que el usuario encontró relevantes
recommended_items = ['Game A', 'Game B', 'Game C', 'Game D']
relevant_items = ['Game A', 'Game C', 'Game E']

precision, recall = precision_recall(recommended_items, relevant_items)

print(f'Precisión: {precision}')
print(f'Recall: {recall}')


Precisión: 0.5
Recall: 0.6666666666666666
